In [3]:
from config import gkey
import gmaps
import numpy as np
import pandas as pd
import requests

In [4]:
analysisdata = pd.read_csv("analysisdata.csv")[["Country","2016","Disaster rate","Internet Access Rate"]]
analysisOECDdata = pd.read_csv("analysisOECDdata.csv")[["Country","2016","Disaster rate","Internet Access Rate","Average Sleeping (Hours)"]]
location= pd.read_csv("../Lisa Code/Happy_Coord2.csv")[["Country","Latitude","Longitude"]]
location.columns=[["Country","Lat","Lng"]]
location.to_csv("coordinate.csv")
newlocation = pd.read_csv("coordinate.csv")[["Country","Lat","Lng"]]
analysisdata_location = pd.merge(analysisdata,newlocation,on = "Country")

In [23]:
gmaps.configure(api_key = gkey)
locations = analysisdata_location[["Lat","Lng"]]
happiness = analysisdata_location["2016"]
internet = analysisdata_location["Internet Access Rate"]
disaster = analysisdata_location["Disaster rate"]
analysisdata_location

,Country,2016,Disaster rate,Internet Access Rate,Lat,Lng
0,Finland,7.41,2.21,0.878286,61.924110,25.748151
1,Norway,7.50,2.19,0.976732,60.472024,8.468946
2,Denmark,7.53,2.89,0.966946,56.263920,9.501785
3,Iceland,7.50,1.52,0.973718,64.963051,-19.020835
4,Switzerland,7.51,2.37,0.894370,46.818188,8.227512
5,Netherlands,7.34,8.24,0.901828,52.132633,5.291266
6,Canada,7.40,3.01,0.916153,56.130366,-106.346771
7,New Zealand,7.33,4.55,0.878601,-40.900557,174.885971
8,Sweden,7.29,2.12,0.888780,60.128161,18.643501
9,Australia,7.31,4.22,0.880014,-25.274398,133.775136


In [28]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=happiness, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 2)
happiness_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)',
                                    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
                                    info_box_content=[f"Happiness Score: {score}" for score in happiness])
fig.add_layer(heat_layer)
fig.add_layer(happiness_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
happiness_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Happiness Score: {score}" for score in happiness]
)
fig = gmaps.figure()
fig.add_layer(happiness_layer)

fig

Figure(layout=FigureLayout(height='420px'))